In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
df = pd.read_csv('Online_Courses.csv')
def extract_numeric_duration(duration_string):
    try:
        parts = duration_string.split()
        numeric_value = float(parts[0])
        if 'hour' in duration_string:
            return numeric_value
        elif 'min' in duration_string:
            return numeric_value / 60
        else:
            return 0
    except:
        return 0

df['Duration'] = df['Duration'].apply(extract_numeric_duration)
features = ['Title', 'Short Intro', 'Category', 'Sub-Category', 'Language', 'Skills']
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Short Intro'].fillna(''))
model = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
model.fit(X)
def recommend_courses(query):
    subject_courses = df[df['Category'].str.lower() == query.lower()]
    if len(subject_courses) > 0:
        X_subject = vectorizer.transform(subject_courses['Short Intro'].fillna(''))
        model_subject = NearestNeighbors(n_neighbors=5, algorithm='brute', metric='cosine')
        model_subject.fit(X_subject)
        random_course = subject_courses.sample(1)
        course_id = random_course['Course_ID'].values[0]
        query_course = subject_courses[subject_courses['Course_ID'] == course_id][features].values.reshape(1, -1)
        distances, indices = model_subject.kneighbors(query_course)
        recommended_course_ids = subject_courses.iloc[indices[0]]['Course_ID'].tolist()
        if course_id in recommended_course_ids:
            recommended_course_ids.remove(course_id)
        recommended_courses = df[df['Course_ID'].isin(recommended_course_ids)][['Title', 'URL']]
        return recommended_courses
    else:
        title_matches = df[df['Title'].str.contains(query, case=False)]
        if not title_matches.empty:
            return title_matches[['Title', 'URL']]
        else:
            return None
if __name__ == "__main__":
    c=0
    while True:
        query = input("Enter a subject or title/keywords to get course recommendations (or 'exit' to quit): ").strip().capitalize()
        if query.lower() == 'exit':
            print("Exiting program...")
            break
        recommended_courses = recommend_courses(query)
        if recommended_courses is None:
            print(f"No courses found for '{query}'")
        else:
            print(f"\nRecommended Courses for '{query}':")
            for index, row in recommended_courses.iterrows():
                if(c>4):
                  break
                print(f"Title: {row['Title']}")
                print(f"URL: {row['URL']}")
                print()
                c=c+1